In [1]:
library(Matching)
library(tidyverse)

Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,1,1,0,0,0,0,1,0.2804613,1.389779
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,1,1,0,0,0,0,0,0.2244447,1.289399
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602


In [4]:
set.seed(4958)

In [5]:
ps_matches <- Match(Y = full_data$mort_28_day_int, Tr = full_data$echo_int,
                    X = full_data$ps, M = 1, estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE, ties = FALSE)

Warning message in Match(Y = full_data$mort_28_day_int, Tr = full_data$echo_int, :
“replace==FALSE, but there are more (weighted) treated obs than control obs.  Some treated obs will not be matched.  You may want to estimate ATC instead.”

In [6]:
ps_matches_df <-
full_data[ps_matches$index.treated, "icustay_id", drop = FALSE] %>%
mutate(match = full_data[ps_matches$index.control, "icustay_id"]) %>%
rename(trtd = icustay_id, ctrl = match) %>%
mutate(match = 1:n()) %>%
gather("group", "icustay_id", trtd, ctrl) %>%
select(icustay_id, group, match) %>%
arrange(group, match)

head(ps_matches_df)

icustay_id,group,match
262436,ctrl,1
244460,ctrl,2
298177,ctrl,3
290165,ctrl,4
278397,ctrl,5
212022,ctrl,6


In [7]:
# saveRDS(ps_matches, file = file.path(data_dir, "ps_matches.rds"))
data.table::fwrite(ps_matches_df, file.path(data_dir, "ps_matches_df.csv"))

In [8]:
ids <- full_data[unlist(ps_matches[c("index.control", "index.treated")]), "icustay_id"]
length(ids) / 2
head(ids)

[1] 1626

[1] 262436 244460 298177 290165 278397 212022

In [9]:
data.table::fwrite(data.frame(icustay_id = ids), file = file.path(data_dir, "ps_icustay_id"))

In [10]:
summary(ps_matches)


Estimate...  -0.04674 
SE.........  0.015145 
T-stat.....  -3.0861 
p.val......  0.0020277 

Original number of observations..............  6361 
Original number of treated obs...............  3262 
Matched number of observations...............  1626 
Matched number of observations  (unweighted).  1626 

Number of obs dropped by 'exact' or 'caliper'  1636 



In [11]:
outcome <- data.frame(echo_pt = full_data$mort_28_day_int[ps_matches$index.treated],
                      match_pt = full_data$mort_28_day_int[ps_matches$index.control])

In [12]:
head(outcome)

echo_pt,match_pt
0,1
0,0
0,1
0,0
1,0
0,1


In [13]:
tab <- table(outcome$echo_pt, outcome$match_pt, dnn = c("Echo", "Non-Echo"))
tab
tab[2, 1] / tab[1, 2]
paste("95% Confint",
      round(exp(c(log(tab[2, 1] / tab[1, 2]) - qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]),
                  log(tab[2, 1] / tab[1, 2]) + qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]))), 7))
mcnemar.test(tab) # for 1-1 pairs

    Non-Echo
Echo   0   1
   0 869 343
   1 267 147

[1] 0.7784257

[1] "95% Confint 0.6633574" "95% Confint 0.9134541"


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 9.2213, df = 1, p-value = 0.002392


In [14]:
(tab[2, 2] + tab[1, 2]) / sum(tab)
(tab[2, 2] + tab[2, 1]) / sum(tab)

[1] 0.301353

[1] 0.2546125

In [15]:
mortality <- data.frame(echo = c(0, 1),
                        mortality = c((tab[2, 2] + tab[1, 2]) / sum(tab),
                                      (tab[2, 2] + tab[2, 1]) / sum(tab)))
mortality

echo,mortality
0,0.3013530
1,0.2546125


In [16]:
mort_fmt <- mortality %>%
mutate(echo = case_when(
    echo == 0 ~ "Non_echo",
    echo == 1 ~ "Echo"
)) %>%
select(echo, mortality) %>%
spread(echo, mortality) %>%
mutate(Method = "PS Match") %>%
select(Method, Echo, Non_echo)
mort_fmt

Method,Echo,Non_echo
PS Match,0.2546125,0.301353


In [17]:
data.table::fwrite(mort_fmt, file.path(data_dir, "mort_ps.csv"))